In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
image_size = (180, 180)
batch_size = 32

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 38233 files belonging to 16 classes.
Using 7646 files for validation.
Found 38233 files belonging to 16 classes.
Using 30587 files for training.


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(180, 180, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(16)
])

In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [6]:
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.fit(
    train_ds,
    epochs=3,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/3
956/956 [==============================] - 22s 20ms/step - loss: 366.2645 - accuracy: 0.2904 - val_loss: 2.2096 - val_accuracy: 0.3062
Epoch 2/3
956/956 [==============================] - 20s 21ms/step - loss: 2.0755 - accuracy: 0.3015 - val_loss: 1.9752 - val_accuracy: 0.3062
Epoch 3/3
956/956 [==============================] - 20s 21ms/step - loss: 1.9540 - accuracy: 0.3002 - val_loss: 1.9213 - val_accuracy: 0.3126


In [7]:
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(val_ds)
print(predictions[0])
print(np.argmax(predictions[0]))

[0.09376461 0.28401476 0.01256572 0.02966072 0.28373897 0.01820228
 0.0141989  0.01197027 0.01654361 0.01907535 0.02220177 0.0137989
 0.0181219  0.09795422 0.01722398 0.04696406]
1


In [11]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

TypeError: 'BatchDataset' object is not subscriptable